In [1]:
%%writefile eig1.py

import numpy as np
from scipy import linalg


def eig1(A,c,isMax, isSym):
  n = A.shape[0]
  if c is None or c > n:
    c = n

  if isSym:
    A = np.maximum(A,A.T)

  eigvals, eigvecs = np.linalg.eigh(A)

  if isMax == 0:
    idx = np.argsort(eigvals)
  else:
    idx = np.argsort(eigvals)[::-1]
  idx1 = idx[0:c]
  eigval = eigvals[idx1]
  eigvec = eigvecs[:,idx1]
  eigval_full = eigvals[idx]

  return eigvec, eigval, eigval_full

Writing eig1.py


In [2]:
%%writefile ClusteringMeasure.py
import numpy as np
from scipy.optimize import linear_sum_assignment

def MutualInfo(L1, L2):
    """Compute Normalized Mutual Information (NMI) between two clusterings."""
    L1 = L1.flatten()
    L2 = L2.flatten()
    if L1.shape != L2.shape:
        raise ValueError("size(L1) must == size(L2)")

    # shift to start from 0
    L1 = L1 - np.min(L1)
    L2 = L2 - np.min(L2)
    nClass = max(L1.max(), L2.max()) + 1

    # contingency table
    G = np.zeros((nClass, nClass))
    for i in range(nClass):
        for j in range(nClass):
            G[i, j] = np.sum((L1 == i) & (L2 == j)) + np.finfo(float).eps

    sumG = G.sum()

    P1 = G.sum(axis=1) / sumG
    P2 = G.sum(axis=0) / sumG

    H1 = -np.sum(P1 * np.log2(P1 + 1e-12))
    H2 = -np.sum(P2 * np.log2(P2 + 1e-12))

    P12 = G / sumG
    PPP = P12 / (P1[:, None] * P2[None, :])
    PPP[PPP < 1e-12] = 1

    MI = np.sum(P12 * np.log2(PPP))
    return MI / max(H1, H2)


def bestMap(L1, L2):
    """Permute labels of L2 to match L1 as best as possible using Hungarian algorithm."""
    L1 = L1.flatten()
    L2 = L2.flatten()
    if L1.shape != L2.shape:
        raise ValueError("size(L1) must == size(L2)")

    L1 = L1 - np.min(L1)
    L2 = L2 - np.min(L2)
    nClass = max(L1.max(), L2.max()) + 1

    # contingency matrix
    G = np.zeros((nClass, nClass))
    for i in range(nClass):
        for j in range(nClass):
            G[i, j] = np.sum((L1 == i) & (L2 == j))

    row_ind, col_ind = linear_sum_assignment(-G)
    mapping = dict(zip(col_ind, row_ind))
    newL2 = np.array([mapping[label] for label in L2])
    return newL2


def ClusteringMeasure(Y, predY):
    """Compute clustering metrics: ACC, NMI, Purity."""
    Y = Y.flatten().astype(int)
    predY = predY.flatten().astype(int)

    # Purity
    correnum = 0
    for ci in np.unique(predY):
        incluster = Y[predY == ci]
        if incluster.size > 0:
            counts = np.bincount(incluster)
            correnum += counts.max()
    Purity = correnum / len(predY)

    # Accuracy via bestMap
    mapped_pred = bestMap(Y, predY)
    ACC = np.mean(Y == mapped_pred)

    # NMI
    MIhat = MutualInfo(Y, mapped_pred)
    return ACC, MIhat, Purity


Writing ClusteringMeasure.py


In [3]:
%%writefile multigraph.py

import numpy as np
from sklearn.cluster import KMeans
from eig1 import eig1
from ClusteringMeasure import ClusteringMeasure
import os







def distance(F,n,j):
  all = np.zeros(n)
  for u in range(n):
    all[u] = np.linalg.norm(F[j,:]- F[u,:]) ** 2
  return all



def multigraph(X,s,alpha, beta, gama):
  mv=len(X)
  n=X[0].shape[0]
  Z=np.eye(n)
  Zv=np.tile(Z[:,:,np.newaxis],(1,1,mv))
  M=np.tile(Z[:,:,np.newaxis],(1,1,mv))
  c=len(np.unique(s))

  wv=np.ones(mv)/mv

  for i in range(200):
    Z[Z<0]=0
    Z=(Z+Z.T)/2
    Zold = Z
    D = np.diag(Z.sum(axis=1))
    L = D-Z


    F, eigval, eigval_full = eig1(L,c,0,1)


    for i in range(mv):
      f = X[i]
      A = f @ f.T + alpha * np.eye(n) + beta * wv[i] * np.eye(n)
      B = beta * wv[i] * Z + f @ f.T
      Zv[:,:,i] = np.linalg.solve(A, B)

      T = Zv[:,:,i]
      T[T<0] = 0
      T = (T + T.T)/2
      Zv[:,:,i] = T

      wv[i] = 1/(2*np.linalg.norm(Zv[:,:,i]-Z, 'fro'))

      M[:,:,i] = wv[i] * Zv[:,:,i]

    for j in range(n):
      all = distance(F,n,j)
      Z[:,j]=(np.sum(M[:,j,:],1) - (gama * all.T)/(4* beta)) / np.sum(wv)

    rel_change = np.linalg.norm(Z-Zold, 'fro')/ np.linalg.norm(Zold, 'fro')


    if i>5 and rel_change < 1e-3:
      break


  res = np.zeros((10,3))
  for i in range(10):
    km = KMeans(n_clusters = c, n_init = 1, random_state = i, verbose =0, init = 'k-means++')
    km.fit(F)
    actual_ids = km.labels_

    res[i,:] = ClusteringMeasure(actual_ids, s)

  result = np.zeros((3,2))


  result[0,0] = np.mean(res[:,0])
  result[0,1] = np.std(res[:,0])

  result[1,0] = np.mean(res[:,1])
  result[1,1] = np.std(res[:,1])

  result[2,0] = np.mean(res[:,2])
  result[2,1] = np.std(res[:,2])


  return result


Writing multigraph.py


In [4]:
%%writefile run.py

import numpy as np
import h5py
from multigraph import multigraph
import os


def load_mat_file(path):
    with h5py.File(path, "r") as f:
        data_refs = f["data"]
        data = []
        for i in range(data_refs.shape[0]):
            ref = data_refs[i][0]
            arr = np.array(f[ref]).T  # transpose -> (n x d)
            data.append(arr)
        labels = np.array(f["labels"]).squeeze().astype(int)
    return data, labels

def normalize_views(data):
    normed = []
    for X in data:
        dist = np.max(X) - np.min(X)
        m01 = (X - np.min(X)) / dist
        Xn = 2 * m01 - 1
        normed.append(Xn)
    return normed

def run_experiment(mat_file="reuters_1200.mat", out_file="reuters.txt"):
    data, labels = load_mat_file(mat_file)
    data = normalize_views(data)

    para1 = [0.01, 1]
    para2 = [100, 1000, 2000]
    para3 = [0.01]

    with open(out_file, "a") as fout:
        for a in para1:
            for b in para2:
                for g in para3:
                    print(f"\nRunning with alpha={a}, beta={b}, gamma={g}")
                    result = multigraph(data, labels, a, b, g)
                    line = [a, b, g] + result.flatten(order="F").tolist()
                    fout.write("\t".join(map(str, line)) + "\n")
                    print(f'Iteration completed')
                    print("Result:", result)
    print(f"\nResults saved to {out_file}")

if __name__ == "__main__":
    run_experiment("reuters_1200.mat", "reuters.txt")

Writing run.py


In [5]:
!python run.py


Running with alpha=0.01, beta=100, gamma=0.01
Iteration completed
Result: [[2.51166667e-01 6.66666667e-04]
 [1.17902994e-01 8.13376551e-04]
 [8.90500000e-01 6.66666667e-04]]

Running with alpha=0.01, beta=1000, gamma=0.01
Iteration completed
Result: [[1.80000000e-01 0.00000000e+00]
 [1.60501441e-02 3.63878666e-18]
 [9.84166667e-01 0.00000000e+00]]

Running with alpha=0.01, beta=2000, gamma=0.01
Iteration completed
Result: [[1.80000000e-01 0.00000000e+00]
 [1.60501441e-02 3.63878666e-18]
 [9.84166667e-01 0.00000000e+00]]

Running with alpha=1, beta=100, gamma=0.01
Iteration completed
Result: [[0.277      0.02937119]
 [0.11807414 0.01637719]
 [0.8155     0.04093559]]

Running with alpha=1, beta=1000, gamma=0.01
Iteration completed
Result: [[0.34883333 0.01840592]
 [0.18806887 0.00776474]
 [0.64033333 0.0311876 ]]

Running with alpha=1, beta=2000, gamma=0.01
Iteration completed
Result: [[0.29425    0.02962134]
 [0.13226457 0.02316597]
 [0.725      0.07036709]]

Results saved to reuters.t